# Format Batch Transcription Results to CSV

This notebook converts JSON transcription results from the Whisper + GPT-4o-mini pipeline into CSV format for spreadsheet analysis.

## Input
- Folder containing JSON transcription files (default: `../files/transcriptions/batch_whisper_gpt4o/`)

## Output
Two CSV files saved to `../files/reports/`:

1. **Summary CSV** (file-level): One row per audio file with full transcriptions and metadata
   - Columns: filename, duration, language, full_kannada_text, full_romanized_text, segment_count, total_cost_usd, processed_at

2. **Detailed CSV** (segment-level): One row per transcription segment with timestamps
   - Columns: filename, segment_id, start_time, end_time, kannada_text, romanized_text, avg_logprob, no_speech_prob

## Features
- Configurable input folder path
- Error handling for malformed JSON files
- Progress tracking
- Timestamped output files + "latest" versions
- Preview of generated CSVs

In [ ]:
import json
import csv
from pathlib import Path
from datetime import datetime
import pandas as pd
from typing import List, Dict, Any

## Configuration

In [ ]:
# Input folder containing JSON transcription files
INPUT_FOLDER = Path("../files/transcriptions/batch_whisper_gpt4o")

# Output folder for CSV reports
OUTPUT_FOLDER = Path("../files/reports")
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

# Timestamp for output filenames
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

print(f"Input folder: {INPUT_FOLDER}")
print(f"Output folder: {OUTPUT_FOLDER}")
print(f"Timestamp: {TIMESTAMP}")

## Load JSON Files

In [ ]:
def load_json_files(folder_path: Path) -> List[Dict[str, Any]]:
    """
    Load all JSON files from a folder.
    
    Args:
        folder_path: Path to folder containing JSON files
        
    Returns:
        List of dictionaries containing parsed JSON data
    """
    json_files = list(folder_path.glob("*.json"))
    print(f"Found {len(json_files)} JSON files in {folder_path}")
    
    results = []
    errors = []
    
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                results.append({
                    'filename': json_file.name,
                    'data': data
                })
        except Exception as e:
            error_msg = f"Error loading {json_file.name}: {str(e)}"
            errors.append(error_msg)
            print(f"❌ {error_msg}")
    
    print(f"\n✅ Successfully loaded {len(results)} files")
    if errors:
        print(f"⚠️  {len(errors)} files failed to load")
    
    return results

# Load all JSON files
json_data = load_json_files(INPUT_FOLDER)

## Generate Summary CSV (File-Level)

In [ ]:
def generate_summary_csv(json_data: List[Dict[str, Any]], output_path: Path) -> pd.DataFrame:
    """
    Generate summary CSV with one row per file.
    
    Args:
        json_data: List of parsed JSON data
        output_path: Path to save CSV file
        
    Returns:
        DataFrame containing summary data
    """
    rows = []
    
    for item in json_data:
        data = item['data']
        metadata = data.get('metadata', {})
        transcription = data.get('transcription', {})
        segments = data.get('segments', [])
        costs = metadata.get('costs_usd', {})
        metrics = metadata.get('metrics', {})
        
        row = {
            'filename': metadata.get('file', item['filename']),
            'duration_seconds': metadata.get('duration', 0),
            'language': metadata.get('language', 'unknown'),
            'full_kannada_text': transcription.get('text_kannada', ''),
            'full_romanized_text': transcription.get('text_romanized', ''),
            'segment_count': len(segments),
            'total_tokens': metrics.get('total_tokens', 0),
            'whisper_cost_usd': costs.get('whisper_transcription', 0),
            'gpt_cost_usd': costs.get('gpt_transliteration_segments', 0),
            'total_cost_usd': costs.get('total_pipeline', 0),
            'processed_at': metadata.get('processed_at', '')
        }
        rows.append(row)
    
    # Create DataFrame and sort by filename
    df = pd.DataFrame(rows)
    df = df.sort_values('filename').reset_index(drop=True)
    
    # Save to CSV
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"\n✅ Summary CSV saved to: {output_path}")
    print(f"   Rows: {len(df)}")
    print(f"   Columns: {len(df.columns)}")
    
    # Calculate totals
    total_duration = df['duration_seconds'].sum()
    total_cost = df['total_cost_usd'].sum()
    print(f"\n📊 Summary Statistics:")
    print(f"   Total duration: {total_duration:.2f} seconds ({total_duration/3600:.2f} hours)")
    print(f"   Total cost: ${total_cost:.4f}")
    print(f"   Average cost per hour: ${(total_cost / (total_duration/3600)):.4f}")
    
    return df

# Generate summary CSV with timestamp
summary_output = OUTPUT_FOLDER / f"batch_whisper_gpt4o_summary_{TIMESTAMP}.csv"
summary_df = generate_summary_csv(json_data, summary_output)

# Also save as "latest" version for convenience
summary_latest = OUTPUT_FOLDER / "batch_whisper_gpt4o_summary_latest.csv"
summary_df.to_csv(summary_latest, index=False, encoding='utf-8')
print(f"\n✅ Also saved as: {summary_latest}")

## Preview Summary CSV

In [ ]:
# Display first few rows (excluding full text columns for readability)
preview_cols = ['filename', 'duration_seconds', 'language', 'segment_count', 'total_cost_usd', 'processed_at']
print("\n📋 Summary CSV Preview (first 10 rows):")
print(summary_df[preview_cols].head(10).to_string(index=False))

# Show sample of full text columns
print("\n📝 Sample Full Text (first file):")
if len(summary_df) > 0:
    print(f"Kannada (first 100 chars): {summary_df.iloc[0]['full_kannada_text'][:100]}...")
    print(f"Romanized (first 100 chars): {summary_df.iloc[0]['full_romanized_text'][:100]}...")

## Generate Detailed CSV (Segment-Level)

In [ ]:
def generate_detailed_csv(json_data: List[Dict[str, Any]], output_path: Path) -> pd.DataFrame:
    """
    Generate detailed CSV with one row per segment.
    
    Args:
        json_data: List of parsed JSON data
        output_path: Path to save CSV file
        
    Returns:
        DataFrame containing detailed segment data
    """
    rows = []
    
    for item in json_data:
        data = item['data']
        metadata = data.get('metadata', {})
        segments = data.get('segments', [])
        filename = metadata.get('file', item['filename'])
        
        for segment in segments:
            row = {
                'filename': filename,
                'segment_id': segment.get('id', ''),
                'start_time': segment.get('start', 0),
                'end_time': segment.get('end', 0),
                'duration': segment.get('end', 0) - segment.get('start', 0),
                'kannada_text': segment.get('text', ''),
                'romanized_text': segment.get('text_romanized', ''),
                'avg_logprob': segment.get('avg_logprob', None),
                'no_speech_prob': segment.get('no_speech_prob', None)
            }
            rows.append(row)
    
    # Create DataFrame and sort by filename and segment_id
    df = pd.DataFrame(rows)
    df = df.sort_values(['filename', 'segment_id']).reset_index(drop=True)
    
    # Save to CSV
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"\n✅ Detailed CSV saved to: {output_path}")
    print(f"   Rows: {len(df)} segments")
    print(f"   Columns: {len(df.columns)}")
    print(f"   Files covered: {df['filename'].nunique()}")
    
    return df

# Generate detailed CSV with timestamp
detailed_output = OUTPUT_FOLDER / f"batch_whisper_gpt4o_detailed_{TIMESTAMP}.csv"
detailed_df = generate_detailed_csv(json_data, detailed_output)

# Also save as "latest" version for convenience
detailed_latest = OUTPUT_FOLDER / "batch_whisper_gpt4o_detailed_latest.csv"
detailed_df.to_csv(detailed_latest, index=False, encoding='utf-8')
print(f"\n✅ Also saved as: {detailed_latest}")

## Preview Detailed CSV

In [ ]:
# Display first few rows (excluding full text columns for readability)
preview_cols = ['filename', 'segment_id', 'start_time', 'end_time', 'duration', 'avg_logprob', 'no_speech_prob']
print("\n📋 Detailed CSV Preview (first 10 segments):")
print(detailed_df[preview_cols].head(10).to_string(index=False))

# Show sample of text columns
print("\n📝 Sample Segment Text (first segment):")
if len(detailed_df) > 0:
    print(f"Kannada: {detailed_df.iloc[0]['kannada_text']}")
    print(f"Romanized: {detailed_df.iloc[0]['romanized_text']}")

## Summary

Successfully generated two CSV files:

1. **Summary CSV**: File-level aggregation with full transcriptions
2. **Detailed CSV**: Segment-level data with timestamps

Both files are saved with timestamps and as "latest" versions for easy access.

### Next Steps
- Import CSVs into Google Sheets or Excel for analysis
- Use detailed CSV for time-based analysis and quality checks
- Use summary CSV for cost analysis and overview statistics
- Filter segments by confidence scores (avg_logprob, no_speech_prob) to identify potential transcription issues